In [1]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [2]:
from mrq.models import EmbedModel

emb_model = EmbedModel("menadsa/S-BioELECTRA", device=device)

# small smoke test
a = emb_model("text")
assert torch.isclose(a.square().sum().sqrt(), torch.tensor(1).float()).item()

print(emb_model("tumor in the chest") @ emb_model("lungs cancer").T)

tensor([[0.6924]])


### Data loading

In [3]:
from mrq.data import load_data

sample_size = 200 # change to 10000000 to use all data
seed = 1337

data = load_data("medmcqa", sample_size=sample_size, seed=seed)
data1 = load_data("AnonymousSub/MedQuAD_47441_Question_Answer_Pairs", sample_size=sample_size, seed=seed)

Using custom data configuration default
Found cached dataset medmcqa (/Users/Vasilii_Salikov/.cache/huggingface/datasets/medmcqa/default/1.1.0/f2fdfa9ccfbf9d148c0639e6afe3379f3c7e95c4d52d5e68ec1156e5004bd880)


2023-05-28 00:06:06,155 - mrq.data - INFO - Loaded data length: 182822
2023-05-28 00:06:06,263 - mrq.data - INFO - Data length after dropna: 87077
2023-05-28 00:06:07,636 - mrq.data - INFO - Final data length: 200


Using custom data configuration AnonymousSub--MedQuAD_47441_Question_Answer_Pairs-5625072dd394af39
Found cached dataset parquet (/Users/Vasilii_Salikov/.cache/huggingface/datasets/AnonymousSub___parquet/AnonymousSub--MedQuAD_47441_Question_Answer_Pairs-5625072dd394af39/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


2023-05-28 00:06:08,988 - mrq.data - INFO - Loaded data length: 47441
2023-05-28 00:06:08,996 - mrq.data - INFO - Data length after dropna: 16407
2023-05-28 00:06:09,009 - mrq.data - INFO - Final data length: 200


### Retrieval

In [6]:
import numpy as np
from mrq.data import AnswerDB, Query
from mrq.algs import GoldenRetriever

db = AnswerDB(data1["A"].tolist()+data["A"].tolist()).encode(emb_model, tqdm_flag=True)
doge = GoldenRetriever(db)

  0%|          | 0/400.0 [00:00<?, ?it/s]

In [163]:
text = "I broke my leg"

query = Query(text).embed(emb_model)

res = doge.find_it(query, top_k=4)
res

{'answers': array(["Mobility aids help you walk or move from place to place if you are disabled or have an injury. They include       - Crutches    - Canes    - Walkers    - Wheelchairs    - Motorized scooters       You may need a walker or cane if you are at risk of falling. If you need to keep your body weight off your foot, ankle or knee, you may need crutches. You may need a wheelchair or a scooter if an injury or disease has left you unable to walk.     Choosing these devices takes time and research. You should be fitted for crutches, canes and walkers. If they fit, these devices give you support, but if they don't fit, they can be uncomfortable and unsafe.",
        'Your legs are made up of bones, blood vessels, muscles, and other connective tissue. They are important for motion and standing. Playing sports, running, falling, or having an accident can damage your legs. Common leg injuries include sprains and strains, joint dislocations, and fractures.    These injuries can affec